# Lab | Flow Control

Objective: Practice how to use programming constructs like if/else statements and loops to control the flow of a program's execution.

## Challenge: The Haunted Mansion

You are a brave adventurer who has decided to explore the Haunted Mansion, a decrepit old building that is rumored to be haunted by ghosts and spirits. Your objective is to find the treasure that is hidden somewhere in the mansion.

## Requirements

- Your script should have at least two functions: "run_mansion()" and "encounter_ghost()".
- Your script should use if-else statements, while loops, for loops, combined loops, or nested loops to control the flow of execution.
- Your script should prompt the user for input to make decisions on which path to take or what actions to perform.
- Your script should include random events and obstacles that can either help or hinder the adventurer in their quest.
- Your script should have an objective of finding the treasure at the end of the mansion.

## Instructions

- Begin by creating a list of items that the adventurer can pick up along the way. These items will be used to help the adventurer overcome obstacles and defeat ghosts. Examples of items can be weapons, potions, keys, etc.

- Complete the function called "run_mansion()" that serves as the main function for the game. Within "run_mansion()", prompt the user to choose a path to take at each intersection. Each path should have its unique challenges and obstacles that the adventurer must overcome.

- Use loops to check if the adventurer has enough health points to continue the game. If their health points drop to zero, the game is over.

- Complete the function called "encounter_ghost()" that will be used to handle ghost encounters. The function should use random events to determine the outcome of the encounter, and the adventurer should use their items to help them defeat the ghost.

- Use loops to generate random events or items along the way. These events can either help or hinder the adventurer, and the outcome should be based on random chance.

- At the end of the mansion, the adventurer will find the treasure, and the game will end.



*Introduction to Functions*:

    Functions are blocks of code that perform a specific task. They are used to break up complex code into smaller, more manageable parts, which can make your code easier to read and understand. Functions can also be reused multiple times throughout your code, which can save you a lot of time and effort.

    Functions are defined using the def keyword, followed by the name of the function and a set of parentheses. Inside the parentheses, you can list any arguments that the function needs in order to perform its task. These arguments are optional, but they can be useful if you need to pass data into the function from outside.

    Once you have defined a function, you can call it from anywhere in your code using its name and passing any necessary arguments. When the function is called, the code inside it is executed, and any values that it returns are passed back to the calling code.

    In this exercise, we have defined a function called encounter_ghost that simulates a battle between the adventurer and a ghost, and run_mansion. Your task is to complete these functions by using flow control statements such as if, else, while, for loops, and nested loops. Remember to pay attention to the instructions and comments provided in the function to help guide you. Once you have completed the function encounter_ghost, you can call it from the main code to simulate the battle and test your implementation.

    Good luck!

In [535]:
import random as rnd
import time as t

health_potion = {"name": "Health potion", 
                 "desc": "A small potion with a red liquid. Seems like it would be useful if your health goes down to 0...",
                "price": 50}
sword = {"name": "Sword", 
         "desc": "A bladed weapon that increases your chances of survial.",
        "price": 100}
bow = {"name": "Bow", 
       "desc": "A ranged weapon. Useful against enemies that can fly, i guess.",
      "price": 70}
shield = {"name": "Shield", 
          "desc": "A sturdy buckler. Increases your chances of survival.",
         "price": 100}
magic_wand = {"name": "Magic wand","desc": "A small wand of oak wood. It seems to sparkle with magic... i wonder what its for?"}
magic_cape = {"name": "Magic cape", 
              "desc": "A cape made from magical cloth. It seems to be moving by itself and tries to constantly lift you off the ground.",
             "price": 150}

possible_items = [health_potion, sword, bow, shield, magic_wand, magic_cape, "pile of coins"] # weighted: 10 30 20 30 5 30 50
possible_foes = ["a ghost", "rats", "evil bats", "the evil mage", "a burglar", "a merchant"] # weighted: 10 40 20 5 30 10
possible_events = ["talking door", "trap door", "spike trap", "a merchant", "golden door", "empty", "treasure chest"] # weighted: 10 20 30 10 5 50

INIT_GAME_STATE = {
    "items" : set(),
    "life" : 10,
    "money" : 100,
}

In [520]:
def map_moves(moves):

    """
    A not-so-simple function to create a map of "#" based on the number of decisions taken by the player. Starting from a approximated middle
    the players steps are retraced by alternating the handled value (between height and width) and reversing the polarity of the steps
    if two same turns are taken after one another.
    """
    
    map_ = {}
    for height in range(32):
        for width in range(32):
            map_[(height,width)] = "#"
    pos = (16,16)
    map_[pos] = "X"
    xy = 1
    for direction in moves:
        if xy == 1:
            xy = 0
            steps = "ן"
        else:
            xy = 1
            steps = "-"
        if "left" in direction:
            new_pos = list(pos)
            new_pos[xy] += -1
            pos = tuple(new_pos)
            map_[pos] = steps
        elif "right" in direction:
            new_pos = list(pos)
            new_pos[xy] += 1
            pos = tuple(new_pos)
            map_[pos] = steps
    for h in range(32):
        print(" ".join([map_[(h,w)] for w in range(32)]))

def check_up(health, gold, items):

    """
    This function takes in health, gold and items as parameters and returns the current state of the player. It shows all current items,
    health and gold and allows the user to read a description of each item.
    """
    
    print("\nYou take a look at yourself and in your bag.")
    for i in items:
        print(f"---- You currently have a {i}.")
    if len(items) == 0:
        print("---- You currently have no items with you.")
    print(f"\n---- You are currently at {health} health points.")
    print(f"---- In your bags you have {gold} golden coins.\n")
    user_input = input(f"Would you like to take a closer look at anything? Type 'check' followed by the item.")
    if "check" in user_input.lower():
        for i in possible_items[:-1]:
            if i["name"].lower() in user_input.lower():
                print(i["desc"])
        if len(items) == 0:
            print("---- You currently have no items with you.")
        t.sleep(2)

In [536]:
def encounter(enemy, health, gold, items):
    """
    This function handles the encounter with each enemy. Each enemy has different chances of being defeated, handled by random module and a number
    between 1 and 10. Sword and Shield items factor into this as an additional +1 to the outcome respectively. 
    """

    print(f"You encounter {enemy}!")
    upgrade = 0
    for up in ["Sword","Shield"]:
        if up in items:
            upgrade += 1
    
    # Introduces the Ghost, an serious enemy that curses the player if defeated and deals 2 damage if not. Also has multiple lifes.
    
    if enemy == "a ghost":
        print("You begin fighting with the ghost!\n")
        t.sleep(2)
        outcome = rnd.randint(1,11)
        if outcome <= 4:
            print("With a low howl the ghost vanishes in the shadows of the mansion!")
            second_step = input("\nTwo paths lead through the mansions spooky walls. Pick a side, left? Or right? You can also check your inventory.\n You go")
            if "left" in second_step or "right" in second_step:
                print(f"As you turn to the {second_step} path a ghostly hand suddenly grabs you from behind and jerks you back! The ghost is back!")
            elif "check" in second_step:
                print(f"As you turn to {second_step} your inventory a ghostly hand suddenly grabs you from behind and jerks you back! The ghost is back!")
            health -= 1
            print(f"You immediately loose one health as you begin to fight the ghost again! You are now at {health} health...\n")
            t.sleep(2)
            outcome = rnd.randint(1,11)
            if outcome <= 4:
                print("You have finally vanquished the ghost! Praise the hero!")
            else:
                health -=2
                print(f"Oh no! You lost to a ghost! You loose two of your healthpoints and are now left with {health}.")
        else:
            health -= 3
            print(f"Oh no! You lost to a ghost! You loose three of your healthpoints and are now left with {health}.")
            print("What a pity... another ghost will soon join the haunt.")

    # Introduces the Rats, an enemy that drops some money if defeated but can potentially destroy an item or some gold if not.
    
    elif enemy == "rats":
        print("A horde of rabid rats attacks you from the shadows!")
        print("In a frenzy they start attacking you!\n")
        t.sleep(2)
        outcome = rnd.randint(1,11)+upgrade
        if outcome <= 6:
            print("With wild swiping attacks you fight off the rats! Praise the exterminator!")
            print("Amidst the retreating masses you see some coins!")
            added_gold = rnd.randrange(5,50,5)
            gold += added_gold
            print(f"You add {added_gold} gold to your bag. You are now at {gold} gold.")
        elif outcome == 7:
            if len(items) != 0:
                possible_choices = [i for i in items]
                random_item = rnd.choices(possible_choices)[0]
                print(f"More and more rats ambush you! In your panic you drop your {random_item}.")
                items.remove(random_item)
            elif gold > 25:
                drop_gold = rng.randrange(5,25,5)
                print(f"More and more rats ambush you! In your panic you drop {drop_gold} of your coins.")
            else:
                health -= 1
                print(f"The rats overwhelm you, biting fiercly. You loose one health point and are now at {health}.")
        else:
            health -= 1
            print(f"The rats overwhelm you, biting fiercly. You loose one health point and are now at {health}.")    

    # Introduces the Evil Bats, an enemy that is easy to defeat but has multiple lives. The bow is helpful in defeating them.
    
    elif enemy == "evil bats":
        amount_of_bats = rnd.randint(1,6)
        print(f"You see a swarm of {amount_of_bats} devilish red bats hover in front of you! Luckily theyre attacking you one by one!")
        t.sleep(2)
        if "Bow" in items:
            if amount_of_bats > 2:
                hits = rnd.randint(1,2)
            else:
                hits = 1
            print(f"With your bow you manage to shoot down {hits} of the bats before they reach you!")
            amount_of_bats -= hits
            t.sleep(1)
        while amount_of_bats != 0:
            outcome = rnd.randint(1,11)+upgrade
            if outcome <= 5:
                amount_of_bats -= 1
                print(f"With a quick strike you kill a bat! {amount_of_bats} bats left!")
                t.sleep(1)
            else:
                print("A bat bites you, deals 1 damage and takes off. It seems some blood is enough to satisfy its hunger.")
                amount_of_bats -= 1
                print(f"{amount_of_bats} bats left!")
                health -= 1
                print(f"You are now at {health} health")
                t.sleep(1)

    # Introducing the evil Mage, a small boss that owns the haunted mansion. Its a multistage enemy that can be weakened by
    # having the magic wand, a rare artefact.
    
    elif enemy == "the evil mage":
        if "golden key" in items:
            print("You come across the corpse of a dark mage... it is time to leave this place..")
            return health, gold
        boss_health = 7
        print("A tall man in a dark robe stands before you. In his hand you see a broken staff, as if half of it is missing...")
        print("\"OH? AND WHO MIGHT YOU BE? WHAT DO YOU THINK YOUR DOING IN MY MANSION?..\"\n He turns to you and points his staff at you.")
        t.sleep(2)
        if "Magic wand" in items:
            print("\nSuddenly the magic wand in your own backpack explodes in magic energy, dispelling the energy beam shot at you.")
        else:
            health -= 2
            print(f"A beam of black energy hits you square in the chest, dealing two damage and leaving you at {health} health points.")

        while health >= 0:
            user_input = input("You see the ire in his eyes as he directs his staff to fire another beam. Where do you want to dodge? (left/right)\n")
            chance = rnd.randint(1,2)
            if "left" in user_input.lower() and chance == 1:
                damage = rnd.randint(2,3)
                health -= damage
                print(f"You tried to avoid it, but the beam still manages to damage you! You now have {health}")
            elif "right" in user_input.lower() and chance == 2:
                damage = rnd.randint(2,3)
                health -= damage
                print(f"You tried to avoid it, but the beam still manages to damage you! You now have {health}")
            else:
                print("You manage to dodge the attack! Your time to strike back!")
                damage = 1 +upgrade
                print(f"With a swift slash you deal the mage {damage} damage, driving him back!")
                boss_health -= damage
            if boss_health <= 0:
                print("You managed to defeat the evil mage! You found a key in the ghostly remains!\nYou can now open the door to the mansion and escape!")
                items.add("golden key")
                break

    # Introducing the burglar, a simple enemy that drops a health potion, bow or some money if defeated but will take a substantial amount
    # of money if he wins a fight.
    
    elif enemy == "a burglar":
        print("You surprise the masked man with a bag over his shoulder! He seems to be looting a body as you startle him.")
        print("\"YOU! What are you doing here! I wont let you get away!\"")
        print("With a yelp and an old dagger he charges you and you begin to fight!\n")
        t.sleep(2)
        outcome = rnd.randint(1,11)+upgrade
        if outcome <= 7:
            item = rnd.choices(possible_items, weights=[30,0,20,0,0,0,30])[0]
            print("Your skills are greater than his, and with a quick slash you strike him down.")
            if item == "pile of coins":
                print(f"In his bag you find a {item}.")
                coins = rnd.randrange(25,50,25)
                print(f"You stash the {coins} golden coins in your bag.")
                if gold + coins > 500:
                    print("You fill up your bag with the gold until you have 500 coins. The rest you leave behind for another day...")
                    gold = 500
                elif gold == 500:
                    print("You already have a full bag. You leave the gold for another day...")
                else:
                    gold += coins
                    print(f"You now have {gold} gold coins.")
            else:
                print(f"In his bag you find a {item['name']}.")
                if item["name"] in items:
                    print(f"You already have a {item['name']}, unfortunately you cant carry more. Maybe the next adventurer will need this...")
                items.add(item["name"])
        else:
            print("His sneaky attacks take you by surprise and you loose the fight!")
            print("\"Harhar! You were a worthy victim, so i will let you go this time! BUT. You have to give me your gold!\"")
            try:
                loss = rnd.randrange(10,gold,25)
                print(f"He takes most of your gold and leaves to hide back in the winding paths of the mansion. You loose {loss} gold.")
                gold -= loss
            except ValueError:
                print("The thief notices your empty pockets and with a pitiful look he turns away from you and leaves.")

    elif enemy == "a merchant":
        health, gold = merchant(health, gold, items)

    return health, gold

In [526]:
def talking_door(health,items):
    goal = rnd.randint(1,health)
    # print(f"The door is thinking about {goal}")
    door_riddle = input(f"I am magical door! To pass me, you must guess my thoughts! Haha, what number am i thinking off?\n(Enter a number between 1 and {health})")
    if door_riddle == str(goal):
        print("\"How...how did you know! Aaaargh!!\"\nThe door breaks open, revealing a healing potion behind it, which you pick up.")
        if "Health potion" in items:
            print("You find a health potion... but you already have one, so you leave it behind.")
        else:
            items.add(health_potion["name"])
    else:
        print("\"Ehehehe, that is not correct!\"\nA cold wind throws you to the floor as the door fades into the wall.")
        print(f"You feel a sudden pang of pain and are now left with {health-1}")
        health -= 1
    return health
    
def trap_door(health, gold, items):
    print("You feel the ground open up underneath you! You start falling!\n")
    t.sleep(2)
    if "Magic cape" in items:
        print("Suddenly your magical cape comes alife and carries you into the air! You are safe...for now.")
        print("As you reach the surface again, your cape disintegrates...")
        items.remove("Magic cape")
    else:
        health -= 3
        print(f"You slam on the ground and feel your bones break... you are now left with {health} health points")
        if health <= 0:
            pass
        else:
            enemy = rnd.choices(possible_foes, weights = [10,40,20,5,30,0])[0]
            print(f"Before you... stands an enemy!")
            health, gold = encounter(enemy, health, gold, items)
    return health, gold
    
def spike_trap(health):
    damage = rnd.randint(1,3)
    print(f"Ouch! Vicious spikes erupt from the floor underneath you! You loose {damage} health points!")
    health -= damage
    print(f"You are now at {health} health points.")
    return health

def merchant(health, gold, items):
    print("\nYou come across a tall spindly man standing next to a booth.\n\"Hello there, stranger! You seem to be in need of some.. support?\"")
    print("As he turns around you notice several items on the desk of the booth.\n")
    wares = rnd.choices(possible_items, weights=[30,30,20,10,0,30,0], k=3)
    wares_dict = {item["name"]:item["price"] for item in wares}
    print(f"\"I can offer you today:\"")
    for i in wares_dict:
        print(f"{i} for {wares_dict[i]} gold.")
    while True:
        user_input = input(f"\n\"Which item would you like to purchase? If you would like to check an item first, just tell me.\"\n You currently have {gold} coins. (type buy or check item)")
        if "check" in user_input.lower():
            for i in possible_items[:-1]:
                if i["name"].lower() in user_input.lower():
                    print(i["desc"])
                    user_input_leave = input(" ---- Would you like to leave?")
                    if "yes" in user_input_leave.lower():
                        return health, gold
        elif "buy" in user_input.lower():
            for i in possible_items[:-1]:
                if i["name"].lower() in user_input.lower() and i["name"] in wares_dict.keys():
                    if gold >= wares_dict[i["name"]]:
                        gold -= wares_dict[i["name"]]
                        print(f"You take the {i['name']} and are left with {gold} gold coins.")
                        print("\"Wise choice! Help yourself...\"")
                        if i["name"] == "Health potion":
                            print(f"You drink the potion, healing you by two health points. You are now at {health} health points.")
                        else:
                            items.add(i["name"])
                        user_input_leave = input(" ---- Would you like to leave?")
                        if "yes" in user_input_leave.lower():
                            return health, gold
                    else:
                        print("\"Im afraid you do not have the coin for it...\"")   
                        user_input_leave = input(" ---- Would you like to leave?")
                        if "yes" in user_input_leave.lower():
                            return health, gold
        else:
            print("\"Suit yourself then. Maybe next time...\"")
            break
    return health, gold

def golden_door(health, gold, items):
    offer = input("You see golden door before you. It seems to be closed, do you want to try and open it anyway? (yes/no)")
    if "yes" in offer.lower():
        if "golden key" in items:
            print("You manage to open the golden door and escape the spooky mansion!")
            return True
        else:
            print("Unfortunately it seems like you need an equally golden key to open the door.")
            return False
    else:
        return False

def treasure(gold,items):
    item = rnd.choices(possible_items, weights=[10,30,20,30,5,30,20])[0]
    if item != "pile of coins":
        print(f"You find a treasure chest! Inside, there is a {item['name']}!")
        if item["name"] in items:
            print(f"You already have a {item['name']}, unfortunately you cant carry more. Maybe the next adventurer will need this...")
        else:
            print(f"You put the {item['name']} in your bag.")
            items.add(item["name"])
    else:
        coins = rnd.randrange(25,100,25)
        print(f"You find a pile of gold on the floor! Lucky you! You stash the {coins} golden coins in your bag")
        if gold + coins > 500:
            print("You fill up your bag with the gold until you have 500 coins. The rest you leave behind for another day...")
            gold = 500
        elif gold == 500:
            print("You already have a full bag. You leave the gold for another day...")
        else:
            gold += coins
            print(f"You put the coins in your bag. You now have {gold} gold coins")
    return gold

In [538]:
# main function for the game
def run_mansion(health,gold,items):
    
    print("Welcome to the Haunted Mansion!")
    
    """
    The main function to run the game. Initializes the player health, gold and items and handles the players decision on which path to take. 
    Also controls health potion and map mechanic.
    """
    moves = []
    while True:
        if health <= 0:
            if "Health potion" in items:
                print("Phew, luckily you had a health potion in your backpack...")
                items.remove("Health potion")
                health = 2
                print("You are now back to 2 health. Continue your journey!")
            else:
                print("Game over, you lost all your health points...")
                print("Here is your map of the mansion:\n\n")
                map_moves(moves)
                break
        choose_side = input("\nTwo paths lead through the mansions spooky walls. Pick a side, left? Or right? You can also check your inventory.\n You go")
        if "left" in choose_side.lower():
            moves.append("left")
            chance = 1
            # chance = rnd.choices([1,2,3], weights=[50,30,10])
            if chance == 1:
                event = rnd.choices(possible_events, weights=[50,20,30,20,5,50,40])[0]
                # print(event+"\nHP: "+str(health)+"\nGold: "+str(gold)+f"\nItems: {items}\n")
                if event == "talking door":
                    health = talking_door(health,items)  
                elif event == "trap door":
                    health, gold = trap_door(health, gold, items)
                elif event == "spike trap":
                    health = spike_trap(health)
                elif event == "a merchant":
                    health, gold = merchant(health, gold, items)
                elif event == "golden door":
                    if golden_door(health, gold, items):
                        print("Finally, freedom!")
                        print("Here is your map of the mansion:\n\n")
                        map_moves(moves)
                        break
                    else:
                        enemy = rnd.choices(possible_foes, weights = [10,40,20,5,30,0])[0]
                        print(f"As you look upon the closed door, an enemy appears and attacks you!")
                        health, gold = encounter(enemy, health, gold, items)
                elif event == "treasure chest":
                    gold = treasure(gold,items)
                else:
                    print("Hmm... seems like there is nothing here...")
            else:
                print("You found a health potion!")
                items.add("health potion")
        elif "right" in choose_side.lower():
            moves.append("right")
            random_enemy = rnd.choices(possible_foes, weights = [10,40,20,5,30,10])[0]
            health, gold = encounter(random_enemy, health, gold, items)
        elif "check" in choose_side.lower() or "inventory" in choose_side.lower():
            check_up(health, gold, items)
        elif "death" in choose_side.lower():
            health -= 10

To run the game, simply call the run_mansion() function:

In [534]:
game_state = INIT_GAME_STATE.copy()

run_mansion(game_state["life"],game_state["money"],game_state["items"])

Welcome to the Haunted Mansion!



Two paths lead through the mansions spooky walls. Pick a side, left? Or right? You can also check your inventory.
 You go left


Ouch! Vicious spikes erupt from the floor underneath you! You loose 2 health points!
You are now at 8 health points.



Two paths lead through the mansions spooky walls. Pick a side, left? Or right? You can also check your inventory.
 You go right


You encounter rats!
A horde of rabid rats attacks you from the shadows!
In a frenzy they start attacking you!

The rats overwhelm you, biting fiercly. You loose one health point and are now at 7.


...


Two paths lead through the mansions spooky walls. Pick a side, left? Or right? You can also check your inventory.
 You go right


You encounter rats!
A horde of rabid rats attacks you from the shadows!
In a frenzy they start attacking you!

The rats overwhelm you, biting fiercly. You loose one health point and are now at 3.



Two paths lead through the mansions spooky walls. Pick a side, left? Or right? You can also check your inventory.
 You go right


You encounter the evil mage!
You come across the corpse of a dark mage... it is time to leave this place..


KeyboardInterrupt: Interrupted by user

This should print the game's narrative and prompt the user to make choices and fight ghosts. The game ends when the adventurer finds the key or loses all their health points. 